In [1]:
#!pip install eeglib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from eeglib.helpers import Helper
from eeglib.eeg import EEG

In [2]:
def df_to_eeg_helper(df, columns, window_size, high_pass=50.0, low_pass=1.0, normalize=False):
    data = df[columns].copy().to_numpy().transpose()
    helper = Helper(
        data, 
        sampleRate=200, 
        names=columns, 
        windowSize=window_size,
        highpass=high_pass,
        lowpass=low_pass,
        normalize=normalize
    )
    return helper

In [3]:
eeg_df = pd.read_csv("train_eeg/175001942_filtered.csv", index_col=0)

In [4]:
helper = df_to_eeg_helper(eeg_df.iloc[4000:6000, :], list(eeg_df.columns), window_size=2000, low_pass=None, high_pass=None)

In [5]:
EEG = [eeg for eeg in helper][0]

In [6]:
EEG

In [31]:
bp = EEG.bandPower()

In [9]:
EEG.PSD().shape

(18, 501)

In [10]:
def get_features(eeg):
    
    bp = eeg.bandPower()
    # Split band power and convert to db
    bp_alpha = [10.0 * np.log10(ch["alpha"]) for ch in bp]
    bp_beta = [10.0 * np.log10(ch["beta"]) for ch in bp]
    bp_delta = [10.0 * np.log10(ch["delta"]) for ch in bp]
    bp_theta = [10.0 * np.log10(ch["theta"]) for ch in bp]
    psd = eeg.PSD()
    hfd = list(eeg.HFD())
    hjorth_mobility = list(eeg.hjorthMobility())
    hjorth_complexity = list(eeg.hjorthComplexity())
    dfa = list(eeg.DFA())
    sampEn = list(eeg.sampEn())

    df_as_dict = {}
    #print(bp_alpha)
    for chan_idx, col_name in enumerate(helper.names):
        df_as_dict[f"{col_name}.abs_bp_alpha"] = [bp_alpha[chan_idx]]
        df_as_dict[f"{col_name}.abs_bp_beta"] = [bp_beta[chan_idx]]
        df_as_dict[f"{col_name}.abs_bp_delta"] = [bp_delta[chan_idx]]
        df_as_dict[f"{col_name}.abs_bp_theta"] = [bp_theta[chan_idx]]
        
        total_bp = bp_alpha[chan_idx]+bp_beta[chan_idx]+bp_delta[chan_idx]+bp_theta[chan_idx]
        df_as_dict[f"{col_name}.total_bp"] = [total_bp]
        df_as_dict[f"{col_name}.rel_bp_alpha"] = [bp_alpha[chan_idx]/total_bp]
        df_as_dict[f"{col_name}.rel_bp_beta"] = [bp_beta[chan_idx]/total_bp]
        df_as_dict[f"{col_name}.rel_bp_delta"] = [bp_delta[chan_idx]/total_bp]
        df_as_dict[f"{col_name}.rel_bp_theta"] = [bp_theta[chan_idx]/total_bp]
        
        #[delta/alpha, delta/beta, delta/theta, theta/alpha, theta/beta, alpha/beta, beta/(alpha+theta), and theta/(alpha+beta)
        df_as_dict[f"{col_name}.bp_delta_alpha"] = [bp_delta[chan_idx]/bp_alpha[chan_idx]]
        df_as_dict[f"{col_name}.bp_delta_beta"] = [bp_delta[chan_idx]/bp_beta[chan_idx]]
        df_as_dict[f"{col_name}.bp_delta_theta"] = [bp_delta[chan_idx]/bp_theta[chan_idx]]
        df_as_dict[f"{col_name}.bp_theta_alpha"] = [bp_theta[chan_idx]/bp_alpha[chan_idx]]
        df_as_dict[f"{col_name}.bp_theta_beta"] = [bp_theta[chan_idx]/bp_beta[chan_idx]]
        df_as_dict[f"{col_name}.bp_alpha_beta"] = [bp_alpha[chan_idx]/bp_beta[chan_idx]]
        df_as_dict[f"{col_name}.bp_beta_alpha+theta"] = [bp_beta[chan_idx]/(bp_alpha[chan_idx]+bp_theta[chan_idx])]
        df_as_dict[f"{col_name}.bp_theta_alpha+beta"] = [bp_theta[chan_idx]/(bp_alpha[chan_idx]+bp_beta[chan_idx])]
        
        df_as_dict[f"{col_name}.hfd"] = [hfd[chan_idx]]
        df_as_dict[f"{col_name}.hjorth_mobility"] = [hjorth_mobility[chan_idx]]
        df_as_dict[f"{col_name}.hjorth_complexity"] = [hjorth_complexity[chan_idx]]
        df_as_dict[f"{col_name}.dfa"] = [dfa[chan_idx]]
        df_as_dict[f"{col_name}.samp_en"] =[sampEn[chan_idx]]

    df = pd.DataFrame.from_dict(df_as_dict)
    return df.copy(), psd

In [11]:
features, psd = get_features(EEG)

In [12]:
features

,Fp1 - F7.abs_bp_alpha,Fp1 - F7.abs_bp_beta,Fp1 - F7.abs_bp_delta,Fp1 - F7.abs_bp_theta,Fp1 - F7.total_bp,Fp1 - F7.rel_bp_alpha,Fp1 - F7.rel_bp_beta,Fp1 - F7.rel_bp_delta,Fp1 - F7.rel_bp_theta,Fp1 - F7.bp_delta_alpha,...,Cz - Pz.bp_theta_alpha,Cz - Pz.bp_theta_beta,Cz - Pz.bp_alpha_beta,Cz - Pz.bp_beta_alpha+theta,Cz - Pz.bp_theta_alpha+beta,Cz - Pz.hfd,Cz - Pz.hjorth_mobility,Cz - Pz.hjorth_complexity,Cz - Pz.dfa,Cz - Pz.samp_en
0,32.5181,37.898691,37.508281,35.562963,143.488035,0.226626,0.264124,0.261404,0.247846,1.153459,...,1.005905,0.928888,0.923435,0.539863,0.482932,2.063673,0.799555,1.181537,0.797024,1.516725
